In [4]:
#Implementing decision tree
class Decisiontree():
    def __init__(self):
        import numpy
        import csv
        self.np = numpy
        self.csv = csv
        self.is_continous = [1,0,1,0,1,0,0,0,0,0,1,1,1,0]
        self.values_num = [] #number of values under given attribute
        self.discrete = {1:['Private', 'Self-emp-not-inc', 'Self-emp-inc', 'Federal-gov', 'Local-gov', 'State-gov', 'Without-pay', 'Never-worked'],
         3:['Bachelors', 'Some-college', '11th', 'HS-grad', 'Prof-school', 'Assoc-acdm', 'Assoc-voc', '9th', '7th-8th', '12th', 'Masters', '1st-4th', '10th', 'Doctorate', '5th-6th', 'Preschool'],
        5:['Married-civ-spouse', 'Divorced', 'Never-married', 'Separated', 'Widowed', 'Married-spouse-absent', 'Married-AF-spouse'],
        6:['Tech-support', 'Craft-repair', 'Other-service', 'Sales', 'Exec-managerial', 'Prof-specialty', 'Handlers-cleaners', 'Machine-op-inspct', 'Adm-clerical', 'Farming-fishing', 'Transport-moving', 'Priv-house-serv', 'Protective-serv', 'Armed-Forces'],
        7:['Wife', 'Own-child', 'Husband', 'Not-in-family', 'Other-relative', 'Unmarried'],
        8:['White', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other', 'Black'],
        9:['Male', 'Female'],
        13:['United-States', 'Cambodia', 'England', 'Puerto-Rico', 'Canada', 'Germany', 'Outlying-US(Guam-USVI-etc)', 'India', 'Japan', 'Greece', 'South', 'China', 'Cuba', 'Iran', 'Honduras', 'Philippines', 'Italy', 'Poland', 'Jamaica', 'Vietnam', 'Mexico', 'Portugal', 'Ireland', 'France', 'Dominican-Republic', 'Laos', 'Ecuador', 'Taiwan', 'Haiti', 'Columbia', 'Hungary', 'Guatemala', 'Nicaragua', 'Scotland', 'Thailand', 'Yugoslavia', 'El-Salvador', 'Trinadad&Tobago', 'Peru', 'Hong', 'Holand-Netherlands'],
        14:['0','1']}
        self.partitions = {}
        self.tree = {}
    
    def most_frequent(self,data,attr):
        #gets the most frequent value in an attribute
        if(len(data)==0):
            return self.np.random.randint(2)
            
        max_val = 0
        temp_data = self.np.array(data)
        best_value = None
        for value in self.discrete[int(attr)]:
            cnt = self.np.array(temp_data)[:,int(attr)].ravel().tolist().count(value)
            if(cnt > max_val):  
                max_val = cnt
                best_value = value
        return best_value
        
    def discretize(self,data,attr):
        
        #discretize/convert in bins, all the continous data
        for record in data:
            if(record[attr] == '?'):
                continue
            c = int(record[attr])
            m,min_val,max_val = self.partitions[attr]
            if(c < m):
                record[attr] = '0'
            elif(c < 2*m):
                record[attr] = '1'
            elif(c < 3*m):
                record[attr] = '2'
            elif(c<4*m):
                record[attr] = '3'
            else:
                record[attr] = '4'
        return data
            
    def partition(self, data, attr):        
        #getting all the values belonging to that attribute
        temp = self.np.array(data[:,attr]).ravel()
        temp = [int(t) for t in temp]
        
        #getting range for deciding partition size
        max_val = 0
        min_val = 1000000000
        
        for num in temp:
            if(num < min_val):
                min_val = num
            if(num > max_val):
                max_val = num
        m = (max_val-min_val)/5
        self.partitions[attr] = [m,min_val,max_val]
        
        self.discrete[attr] = ['0','1','2','3','4']
        self.discretize(data,attr)
        return data
    
    def handle_exceptions(self,data):
        #assign most frequent value in that attribute for invalid values
        for record in data:
            for i in range(len(record)):
                if(record[i] == '?'):
#                     record[i] = self.most_frequent(data,i)
                    record[i] = str(self.discrete[i][self.np.random.randint(self.values_num[i])])
        return data
    
    def process_train_data(self, data):
        
        final_data = data[:]
        for i in range(len(final_data[1,:-1])):
            if(self.is_continous[i]==1):
                final_data = self.partition(final_data, i)
            self.values_num.append(len(self.discrete[i]))
        final_data = self.handle_exceptions(final_data) 
        return final_data
    
    def process_test_data(self,data):
        final_data = data[:]
        for i in range(len(final_data[1,:-1])):
            if(self.is_continous[i] == 1):
                final_data = self.discretize(final_data,i)
        final_data = self.handle_exceptions(final_data)
        return final_data
    
    def clean_data(self,data,string):
        final_data = data[:]
        if(string == "train"):
            final_data = self.process_train_data(final_data)
        else:
            final_data = self.process_test_data(final_data)
        return final_data
    
    #get entropy
    def get_records(self, data, attr, value):
        
        #gets all the examples which have value as attr attribute value
        temp_data = data[:]
        final_records = [record for record in temp_data if record[int(attr)]==value]
        return final_records
    
    def get_entropy(self, data):
        
        #get enrtropy of whole data
        temp_data = data[:]
        entropy = 0.0
        labels_count = [0,0]
        
        labels_count[0] = self.np.array(temp_data)[:,len(temp_data[0])-1].ravel().tolist().count('0')
        labels_count[1] = self.np.array(temp_data)[:,len(temp_data[0])-1].ravel().tolist().count('1')
        
        for i in range(2):
            if(labels_count[i]==0):
                continue
            entropy += ((-1.0*float(labels_count[i])/float(len(data)))*(self.np.log2(float(labels_count[i])) - self.np.log2(float(len(data)))))
        return entropy
    
    def get_gain(self, data, attr):
        
        #gets gain in information if split on attr attribute
        temp_data = data[:]
        gain = 0.0
        subset_entropy = 0.0
        
        for value in self.discrete[int(attr)]:
            data_subset = self.get_records(temp_data, attr, value)
            if(len(data_subset)==0):
                continue
            prob = float(len(data_subset))/float(len(data))
            subset_entropy += prob*self.get_entropy(data_subset)
        gain = self.get_entropy(temp_data) - subset_entropy
        return gain
        
        
    def get_classification(self, record, tree):
        
        #recursice function to traverse the depth of tree(dfs)
        if(type(tree) != dict):
            return tree
        else:
            att = tree.keys()[0]
            t = tree[att][record[int(att)]]
            return self.get_classification(record, t)
     
    def predict(self, file_string):
        raw_data = []
        #read data in form of list of lists
        with open(file_string, 'r') as f:   
            raw = self.csv.reader(f, delimiter = '\n')           
            for row in raw:
                raw_data.append(row[0].split(', '))
        
        test_data = self.clean_data(self.np.array(raw_data),"test")
        temp_data = test_data[:]
        
        classification = []
        for record in temp_data:
            classification.append(self.get_classification(record,self.tree))
        
        return classification
    
    def next_best(self,attributes,data):
        
        #selecting next best attribute for splitting
        best_gain = 0.0
        best_attribute = None
        
        for attr in attributes:
            gain = self.get_gain(data,attr)
            if(gain >= best_gain):
                best_gain = gain
                best_attribute = attr
        return best_attribute
        
    def create_dtree(self ,attributes, data, depth):
        
        #recursive function to create decision tree
        c_data = data[:]
        labels = [record[len(c_data[0])-1] for record in c_data] 
        majority = self.most_frequent(data,14)
        
        if(len(data) == 0 or len(attributes)-1 <= 0 or depth >=10 ):
            return majority
        elif(len(set(labels))==1):
            return labels[0]
        else:
            #getting next best attributeq
            best_attribute = self.next_best(attributes,data)
            #creating a new subtree for next_best_attribute
            tree={best_attribute:{}}
            #create a subtree for every possible value in best_attribute
            for vals in self.discrete[int(best_attribute)]:
                data_subset = self.get_records(c_data, best_attribute, vals)
                attr = [attri for attri in attributes if attri != best_attribute]
                
                next_subtree = self.create_dtree(attr,data_subset,depth+1)
                
                tree[best_attribute][vals] = next_subtree
            return tree
        
    def train(self, file_string):
        raw_data = []
        #read data in form of list of lists
        with open(file_string, 'r') as f:   
            raw = self.csv.reader(f, delimiter = '\n')           
            for row in raw:
                raw_data.append(row[0].split(', '))
        
        attributes = [str(i) for i in range(14)]
        
        train_data = self.clean_data(self.np.array(raw_data), "train")
        
        self.tree = self.create_dtree(attributes,train_data,0)

In [8]:
import dill

model = Decisiontree()
model.train('/home/desmond/Programming/MachineLearning/AML/PA1/DT/dataset.csv')
with open('me15btech11009.model','w') as f:
    dill.dump(model,f)